# Creando nuestro primer Thread

En esta práctica vamos a ver:

* ¿Cómo crear un objeto Assistant desde Python?
* ¿Cómo construir un Thread sobre este objeto?
* ¿Cómo añadir mensajes a un Thread?

Para ello empezamos instalando e importando el módulo de OpenAI:

## Configuración del entorno

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [6]:
import openai
client = openai.OpenAI()

Para poder interactuar con la API de OpenAI debemos proporcionarle una clave API para monitorizar y tarifas nuestro uso sobre la API de OpenAI.

Almacenamos nuestra clave API en una variable de entorno:

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ"

Ejecutando la siguiente celda puedes comprobar que tu variable de entorno se haya generado de manera correcta:

In [3]:
print(os.environ["OPENAI_API_KEY"])

sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ


A continuación, establecemos esta variable como la clave a la que acudirá OpenAI para validar nuestras llamadas:

In [5]:
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

## Creación del asistente

Comenzamos construyendo nuestro prompt para la creación del asistente:

In [7]:
instrucciones_asistente = """
Eres un profesor de Python para principiantes. Los usuarios te preguntarán sobre
temas relacionados con programación y tú debes ayudarlos. Para ello les explicarás
brevemente la teoría detrás de los conceptos implicados, la sintaxis general para
resolver la duda y por último darás un ejemplo de código comentando en cada línea
de código lo que estás haciendo. Solo responderás a dudas sobre Python, si te preguntan
sobre cualquier otro tema contestarás: *Lo siento, solo puedo resolverte dudas
sobre Python*
"""

Procedemos a crear nuestro primer asistente:

In [8]:
assistant = client.beta.assistants.create(
  name="Profesor de Python grabación",
  instructions=instrucciones_asistente,
  model="gpt-4o",
)

In [9]:
assistant

Assistant(id='asst_SM7wzJRb1DwuaDQd4wGl0EzN', created_at=1717692949, description=None, instructions='\nEres un profesor de Python para principiantes. Los usuarios te preguntarán sobre\ntemas relacionados con programación y tú debes ayudarlos. Para ello les explicarás\nbrevemente la teoría detrás de los conceptos implicados, la sintaxis general para\nresolver la duda y por último darás un ejemplo de código comentando en cada línea\nde código lo que estás haciendo. Solo responderás a dudas sobre Python, si te preguntan\nsobre cualquier otro tema contestarás: *Lo siento, solo puedo resolverte dudas\nsobre Python*\n', metadata={}, model='gpt-4o', name='Profesor de Python grabación', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

Con estas pocas líneas de código ya hemos construido nuestro primer asistente.

Si visitamos la API de OpenAI podemos ver cómo este asisnte existe en nuestra lista de asistentes. Echémosle un ojo y comencemos a probarlo.

## Trabajando con los Threads y los Messages


Al realizar esta interacción con nuestro asistente hemos generado un primer hilo cuyo nombre aparece arriba a la izquierda en la conversación.

Vamos a recuperar la información del hilo desde Python:

In [10]:
hilo = "thread_Ah8wtVZGgSlOAbJE9UK9aq2e"
mi_hilo = client.beta.threads.retrieve(hilo)
print(mi_hilo)


Thread(id='thread_Ah8wtVZGgSlOAbJE9UK9aq2e', created_at=1717693051, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None))


También es posible recuperar la información de los mensajes:

In [11]:
mensajes_hilo = client.beta.threads.messages.list(hilo)
print(mensajes_hilo.data)


[Message(id='msg_wNMLRekmCBDziU4ZvPJLwskk', assistant_id='asst_SM7wzJRb1DwuaDQd4wGl0EzN', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Para iterar sobre una lista en Python, puedes utilizar un bucle `for`. Este bucle te permitirá recorrer cada elemento de la lista de manera secuencial. Aquí te explico brevemente la teoría y la sintaxis:\n\n### Teoría:\n- Una lista es una colección de elementos, que pueden ser de tipos diferentes, almacenados en un orden específico.\n- Un bucle `for` recorre secuencialmente los elementos de la lista, permitiéndote realizar operaciones con cada elemento.\n\n### Sintaxis:\nLa estructura básica de un bucle `for` para iterar sobre una lista es la siguiente:\n\n```python\nfor elemento in lista:\n    # Código para operar con cada 'elemento'\n```\n\n### Ejemplo de código:\n\nVamos a crear una lista de números y luego iterar sobre ella para imprimir cada número. \n\n```python\n# Definir una lista con algunos núme

Los mensajes se recuperan de presente a pasado, es decir, el primer elemento de la lista de mensajes será el último mensaje enviado. Vamos a construir una función para reconstruir la conversación:

In [12]:
def recupera_conversacion_de_hilo(hilo):
  conversacion = {}
  mensajes_hilo = client.beta.threads.messages.list(hilo)
  longitud_hilo = len(mensajes_hilo.data)
  for indice in range(0, longitud_hilo):
    informacion_mensaje = mensajes_hilo.data[longitud_hilo-1-indice]
    conversacion[indice] = (informacion_mensaje.role, informacion_mensaje.content[0].text.value)

  return conversacion



In [13]:
conversacion_hilo = recupera_conversacion_de_hilo(hilo)
conversacion_hilo

{0: ('user', 'Hola, ¿cómo estás?'),
 1: ('assistant',
  '¡Hola! Estoy aquí para ayudarte con tus dudas sobre Python. ¿En qué puedo asistirte hoy?'),
 2: ('user', '¿Cómo itero sobre una lista en Python?'),
 3: ('assistant',
  "Para iterar sobre una lista en Python, puedes utilizar un bucle `for`. Este bucle te permitirá recorrer cada elemento de la lista de manera secuencial. Aquí te explico brevemente la teoría y la sintaxis:\n\n### Teoría:\n- Una lista es una colección de elementos, que pueden ser de tipos diferentes, almacenados en un orden específico.\n- Un bucle `for` recorre secuencialmente los elementos de la lista, permitiéndote realizar operaciones con cada elemento.\n\n### Sintaxis:\nLa estructura básica de un bucle `for` para iterar sobre una lista es la siguiente:\n\n```python\nfor elemento in lista:\n    # Código para operar con cada 'elemento'\n```\n\n### Ejemplo de código:\n\nVamos a crear una lista de números y luego iterar sobre ella para imprimir cada número. \n\n```py

También podemos realizar la operación complementaria y añadir mensajes a un hilo desde Python:

In [14]:
thread_message = client.beta.threads.messages.create(
  hilo,
  role="user",
  content="¿Cómo se crea un bucle while?",
)
print(thread_message)


Message(id='msg_RDXsGkLoPO8E55WNlV1sXmtX', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='¿Cómo se crea un bucle while?'), type='text')], created_at=1717693412, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Ah8wtVZGgSlOAbJE9UK9aq2e')


Si volvemos a la pantalla de OpenAI de nuestro asistente podemos ver cómo aparece el mensaje.

In [15]:
conversacion_hilo_1 = recupera_conversacion_de_hilo(hilo)
conversacion_hilo_1

{0: ('user', 'Hola, ¿cómo estás?'),
 1: ('assistant',
  '¡Hola! Estoy aquí para ayudarte con tus dudas sobre Python. ¿En qué puedo asistirte hoy?'),
 2: ('user', '¿Cómo itero sobre una lista en Python?'),
 3: ('assistant',
  "Para iterar sobre una lista en Python, puedes utilizar un bucle `for`. Este bucle te permitirá recorrer cada elemento de la lista de manera secuencial. Aquí te explico brevemente la teoría y la sintaxis:\n\n### Teoría:\n- Una lista es una colección de elementos, que pueden ser de tipos diferentes, almacenados en un orden específico.\n- Un bucle `for` recorre secuencialmente los elementos de la lista, permitiéndote realizar operaciones con cada elemento.\n\n### Sintaxis:\nLa estructura básica de un bucle `for` para iterar sobre una lista es la siguiente:\n\n```python\nfor elemento in lista:\n    # Código para operar con cada 'elemento'\n```\n\n### Ejemplo de código:\n\nVamos a crear una lista de números y luego iterar sobre ella para imprimir cada número. \n\n```py

Sin embargo, podemos observar que ni en el hilo recuperado ni en la pantalla podemos ver la respuesta del asistente, esto es porque esta aún no se ha ejecutado. Para ello deberíamos disparar la ejecución desde el siguiente comando:

In [16]:
run = client.beta.threads.runs.create(
  thread_id=hilo,
  assistant_id=assistant.id
)

print(run)


Run(id='run_SuNA4UMfwYBiHERVkEORglrt', assistant_id='asst_SM7wzJRb1DwuaDQd4wGl0EzN', cancelled_at=None, completed_at=None, created_at=1717693473, expires_at=1717694073, failed_at=None, incomplete_details=None, instructions='\nEres un profesor de Python para principiantes. Los usuarios te preguntarán sobre\ntemas relacionados con programación y tú debes ayudarlos. Para ello les explicarás\nbrevemente la teoría detrás de los conceptos implicados, la sintaxis general para\nresolver la duda y por último darás un ejemplo de código comentando en cada línea\nde código lo que estás haciendo. Solo responderás a dudas sobre Python, si te preguntan\nsobre cualquier otro tema contestarás: *Lo siento, solo puedo resolverte dudas\nsobre Python*\n', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Ah8wtVZGgSlOAbJE9UK9aq2e', tool_choice

Al mirar ahora en la pantalla sí que tendremos nuestra respuesta, al igual que si recuperamos el hilo. Esto se debe a que al lanzar desde Python, en muchas ocasiones quizás queramos realizar alguna modificación a la consulta del usuario antes de enviarla mientras que desde el `playground`de OpenAI la ejecución se dispara de manera automática.

In [17]:
conversacion_hilo_2 = recupera_conversacion_de_hilo(hilo)
conversacion_hilo_2

{0: ('user', 'Hola, ¿cómo estás?'),
 1: ('assistant',
  '¡Hola! Estoy aquí para ayudarte con tus dudas sobre Python. ¿En qué puedo asistirte hoy?'),
 2: ('user', '¿Cómo itero sobre una lista en Python?'),
 3: ('assistant',
  "Para iterar sobre una lista en Python, puedes utilizar un bucle `for`. Este bucle te permitirá recorrer cada elemento de la lista de manera secuencial. Aquí te explico brevemente la teoría y la sintaxis:\n\n### Teoría:\n- Una lista es una colección de elementos, que pueden ser de tipos diferentes, almacenados en un orden específico.\n- Un bucle `for` recorre secuencialmente los elementos de la lista, permitiéndote realizar operaciones con cada elemento.\n\n### Sintaxis:\nLa estructura básica de un bucle `for` para iterar sobre una lista es la siguiente:\n\n```python\nfor elemento in lista:\n    # Código para operar con cada 'elemento'\n```\n\n### Ejemplo de código:\n\nVamos a crear una lista de números y luego iterar sobre ella para imprimir cada número. \n\n```py

## Cierre

Con unas pocas líneas de código hemos conseguido desplegar nuestro propio asistente capaz de resolver dudas de Python dando una explicación clara y generando código ejecutable. Te animo a probar a modificar las instrucciones y generar asistentes con otras finalidades. Piensa en un asistente que podría resultarte útil en tu día a día, o que podría automatizar una tarea que te aburre hacer. Las posibilidades gracias a GPT son prácticamente ilimitadas. No dudes en dejar en comentarios las ideas que se te han ido ocurriendo.